<p style="text-align:center;">
<img src="https://qualitasconsortium.uk/wp-content/uploads/2023/03/nhs-england-logo.png"
     alt="NHSBanner"
     style="float: center; margin-right: 10px;"
     width="680" height="260" />
</p>

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Preparing our Dataframe

### 1.1 Import Data

The Prescription Cost Analysis data is spread across <b>six different datasets</b> - one for each year.

These will be loaded into python individually, and then concatenated to form <b>one dataset containing data for all years</b>. This dataset will then be used for the rest of the analysis.

In [2]:
df_2019 = pd.read_csv('pca_stp_snomed_2019.csv')

In [3]:
df_2020 = pd.read_csv('pca_stp_snomed_2020.csv')

In [4]:
df_2021 = pd.read_csv('pca_stp_snomed_2021.csv')

In [5]:
df_2022 = pd.read_csv('pca_icb_snomed_2022.csv', dtype={5: 'str', 12: 'str'})

In [6]:
df_2023 = pd.read_csv('pca_icb_snomed_2023.csv')

In [7]:
df_2024 = pd.read_csv('pca_icb_snomed_2024.csv')

In [8]:
# Show all columns in a DataFrame
pd.set_option('display.max_columns', None)

# Check that the 2019 dataset has been loaded in properly
df_2019.head();

In [9]:
# Check that the 2024 dataset has been loaded in properly
df_2024.head();

### 1.2 Creating a Single, Usable Dataframe For Our Analysis

All data has now been loaded into separate dataframes. Next, we need to concatenate these together into one dataframe.

* However, we can't do that immediately. This is because upon inspection of each dataframe, the columns vary slightly for the years 2019-2021 and 2022-2024.


* In 2019-2021, the NHS body responsible for the dispensation of prescriptions for a particular defined region were called STP's. In 2022, due to regulatory changes to the NHS, these STP's were 'reformed' to create ICB's. These datasets reflect this change in two of their columns.


* In the 2019-2021 datasets, we have the columns "STP_NAME" and "STP_CODE". In the 2022-2024 datasets, we have "ICB_NAME" and "ICB_CODE".



Before we can concatenate these datasets together, we need to standardise these two column names so they match. We will use the standardised column names: "GEOGRAPHY_NAME" and "GEOGRAPHY_CODE".

Once concatenated, will filter our dataframe to include data relating only to antidepressant prescription.



#### Standardising column names across dataframes

In [10]:
# Rename columns in df_2019, df_2020, and df_2021
df_2019.rename(columns={"STP_NAME": "GEOGRAPHY_NAME", "STP_CODE": "GEOGRAPHY_CODE"}, inplace=True)
df_2020.rename(columns={"STP_NAME": "GEOGRAPHY_NAME", "STP_CODE": "GEOGRAPHY_CODE"}, inplace=True)
df_2021.rename(columns={"STP_NAME": "GEOGRAPHY_NAME", "STP_CODE": "GEOGRAPHY_CODE"}, inplace=True)

# Rename columns in df_2022, df_2023, and df_2024
df_2022.rename(columns={"ICB_NAME": "GEOGRAPHY_NAME", "ICB_CODE": "GEOGRAPHY_CODE"}, inplace=True)
df_2023.rename(columns={"ICB_NAME": "GEOGRAPHY_NAME", "ICB_CODE": "GEOGRAPHY_CODE"}, inplace=True)
df_2024.rename(columns={"ICB_NAME": "GEOGRAPHY_NAME", "ICB_CODE": "GEOGRAPHY_CODE"}, inplace=True)

Let's check if it's worked

In [11]:
df_2021.head(1)

,YEAR_DESC,REGION_NAME,REGION_CODE,GEOGRAPHY_NAME,GEOGRAPHY_CODE,BNF_PRESENTATION_CODE,BNF_PRESENTATION_NAME,SNOMED_CODE,SUPPLIER_NAME,UNIT_OF_MEASURE,GENERIC_BNF_EQUIVALENT_CODE,GENERIC_BNF_EQUIVALENT_NAME,BNF_CHEMICAL_SUBSTANCE_CODE,BNF_CHEMICAL_SUBSTANCE,BNF_PARAGRAPH_CODE,BNF_PARAGRAPH,BNF_SECTION_CODE,BNF_SECTION,BNF_CHAPTER_CODE,BNF_CHAPTER,PREP_CLASS,PRESCRIBED_PREP_CLASS,ITEMS,TOTAL_QUANTITY,NIC,NIC_PER_ITEM,NIC_PER_QUANTITY,QUANTITY_PER_ITEM
0,2021,LONDON,Y56,EAST LONDON HEALTH & CARE P/SHIP STP,QMF,0206020Z0BBACAC,Exforge 10mg/160mg tablets,1.116071e+16,Novartis Pharmaceuticals UK Ltd,tablet,0206020Z0AAACAC,Amlodipine 10mg / Valsartan 160mg tablets,0206020Z0,Valsartan/amlodipine,20602,Calcium-channel blockers,206,"Nitrates, calcium-channel blockers & other ant...",2,Cardiovascular System,3,3,45,1932.0,1829.19,40.64867,0.94679,42.93333


In [12]:
df_2024.head(1)

,YEAR_DESC,REGION_NAME,REGION_CODE,GEOGRAPHY_NAME,GEOGRAPHY_CODE,BNF_PRESENTATION_CODE,BNF_PRESENTATION_NAME,SNOMED_CODE,SUPPLIER_NAME,UNIT_OF_MEASURE,GENERIC_BNF_EQUIVALENT_CODE,GENERIC_BNF_EQUIVALENT_NAME,BNF_CHEMICAL_SUBSTANCE_CODE,BNF_CHEMICAL_SUBSTANCE,BNF_PARAGRAPH_CODE,BNF_PARAGRAPH,BNF_SECTION_CODE,BNF_SECTION,BNF_CHAPTER_CODE,BNF_CHAPTER,PREP_CLASS,PRESCRIBED_PREP_CLASS,ITEMS,TOTAL_QUANTITY,NIC,NIC_PER_ITEM,NIC_PER_QUANTITY,QUANTITY_PER_ITEM
0,2024,LONDON,Y56,NHS NORTH CENTRAL LONDON INTEGRATED CARE BOARD,QMJ,0302000N0BCABAY,Seretide 250 Accuhaler,3.187211e+15,GlaxoSmithKline UK Ltd,dose,0302000N0AAAYAY,Fluticasone propionate 250microg/Salmeterol 50...,0302000N0,Fluticasone propionate (Inhalation),30200,Corticosteroids (respiratory),302,Corticosteroids (respiratory),3,Respiratory System,3,2,732,1138.0,38635.1,52.78019,33.95,1.55464


It has!

Each dataframe is now ready to be concatenated to construct one, new dataframe containing all data. This new dataframe will be used for the rest of the project.

#### Concatenation

In [13]:

# List of DataFrames to Concatenate
list_of_dataframes = [
    df_2019,
    df_2020,
    df_2021,
    df_2022,
    df_2023,
    df_2024
]



In [14]:
# Concatenate all DataFrames in the list
# ignore_index=True will create a new clean index for the combined DataFrame
df = pd.concat(list_of_dataframes, ignore_index=True)

#### Let's verify that the dataframes have been concatenated correctly

In [15]:
# Let's check how many rows are in the dataframe
df.shape

(4618764, 28)

In [16]:
# All the columns look correct!
df.columns.to_list();

In [17]:
# The dataframe begins with 2019 data at index = 0
df.head();

In [18]:
# And ends with 2024 data at index = 4618763
# This all looks correct!
df.tail();

In [19]:
# Let's check how many observations there are for each year
# Matches the original datasets, so all looks good
df['YEAR_DESC'].value_counts().sort_index();

Everything Looks good!

Our dataframe is currently massive (over 4 million rows!). However, we are not concerned with a lot of this data. That is because this dataset covers _all_ prescriptions dispensed in England, not only antidepressant prescriptions.

The last step to getting our dataset ready for cleaning and preparation is to filter it to include _only_ data relating to antidepressant prescriptions.

#### Creating our Antidepressant Dataframe

The column "BNF_SECTION" contains data relating to the drug classification of a prescription.

We first need to check what drug classifications are defined in the dataset

In [20]:
# Check all of the unique classes in the BNF_SECTION field
df['BNF_SECTION'].unique();

We can see that their is only one classification relating to antidepressant prescription, which is called "Antidepressant drugs".

This is what we will use to filter and refine our dataset.

In [21]:
# Create our new dataframe containing data on only antidepressant prescription
df_ad = df[df['BNF_SECTION'] == "Antidepressant drugs"].copy()

In [22]:
# Let's check this new dataset
df_ad.head()

,YEAR_DESC,REGION_NAME,REGION_CODE,GEOGRAPHY_NAME,GEOGRAPHY_CODE,BNF_PRESENTATION_CODE,BNF_PRESENTATION_NAME,SNOMED_CODE,SUPPLIER_NAME,UNIT_OF_MEASURE,GENERIC_BNF_EQUIVALENT_CODE,GENERIC_BNF_EQUIVALENT_NAME,BNF_CHEMICAL_SUBSTANCE_CODE,BNF_CHEMICAL_SUBSTANCE,BNF_PARAGRAPH_CODE,BNF_PARAGRAPH,BNF_SECTION_CODE,BNF_SECTION,BNF_CHAPTER_CODE,BNF_CHAPTER,PREP_CLASS,PRESCRIBED_PREP_CLASS,ITEMS,TOTAL_QUANTITY,NIC,NIC_PER_ITEM,NIC_PER_QUANTITY,QUANTITY_PER_ITEM
2962,2019,LONDON,Y56,EAST LONDON HEALTH & CARE P/SHIP STP,QMF,0403010B0AAA6A6,Amitriptyline 10mg/5ml oral liquid,1.387901e+16,Special Order,ml,0403010B0AAA6A6,Amitriptyline 10mg/5ml oral liquid,0403010B0,Amitriptyline hydrochloride,40301,Tricyclic and related antidepressant drugs,403,Antidepressant drugs,4,Central Nervous System,5,5,10,3000.0,2375.68,237.568000,0.791893,300.000000
2963,2019,LONDON,Y56,EAST LONDON HEALTH & CARE P/SHIP STP,QMF,0403010B0AAAFAF,Amitriptyline 50mg/5ml oral solution sugar free,3.590161e+16,NaN,ml,0403010B0AAAFAF,Amitriptyline 50mg/5ml oral solution sugar free,0403010B0,Amitriptyline hydrochloride,40301,Tricyclic and related antidepressant drugs,403,Antidepressant drugs,4,Central Nervous System,1,1,114,19776.0,2531.32,22.204561,0.128000,173.473684
2964,2019,LONDON,Y56,EAST LONDON HEALTH & CARE P/SHIP STP,QMF,0403010B0AAAFAF,Amitriptyline 50mg/5ml oral solution sugar free,5.637110e+14,Rosemont Pharmaceuticals Ltd,ml,0403010B0AAAFAF,Amitriptyline 50mg/5ml oral solution sugar free,0403010B0,Amitriptyline hydrochloride,40301,Tricyclic and related antidepressant drugs,403,Antidepressant drugs,4,Central Nervous System,5,5,12,4200.0,481.56,40.130000,0.114657,350.000000
2965,2019,LONDON,Y56,EAST LONDON HEALTH & CARE P/SHIP STP,QMF,0403010B0AAAGAG,Amitriptyline 10mg tablets,3.217450e+08,NaN,tablet,0403010B0AAAGAG,Amitriptyline 10mg tablets,0403010B0,Amitriptyline hydrochloride,40301,Tricyclic and related antidepressant drugs,403,Antidepressant drugs,4,Central Nervous System,1,1,169790,7194178.0,237861.59,1.400916,0.033063,42.371035
2966,2019,LONDON,Y56,EAST LONDON HEALTH & CARE P/SHIP STP,QMF,0403010B0AAAGAG,Amitriptyline 10mg tablets,3.109110e+14,A A H Pharmaceuticals Ltd,tablet,0403010B0AAAGAG,Amitriptyline 10mg tablets,0403010B0,Amitriptyline hydrochloride,40301,Tricyclic and related antidepressant drugs,403,Antidepressant drugs,4,Central Nervous System,5,5,16,812.0,26.96,1.685000,0.033202,50.750000


<b>We now have a dataset detailing all antidepressant prescriptions made in England from 2019 to 2024.</b>

We can now move on to data prep + cleaning.

Once the dataframe is clean and ready for EDA, I will also export it as a .csv file.

This is so that it can be imported into Tableau for EDA visualisation, as well as being joined to a shapefile I have for all NHS ICB's across England. This will allow me to visualise the prescription data geographically, using maps in Tableau.

## 2. Data Preparation

### 2.1 Quality Checks

First, let's check for any duplicate entries in the data.

In [23]:
df_ad.duplicated().sum()

0

<b>There are no duplicates!</b>

Now, let's check the shape of the dataset along with some key info relating to it.

In [24]:
df_ad.shape

(77559, 28)

* There are <b>77559 records</b>, and <b>28 columns</b> in the dataset.

In [25]:
df_ad.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77559 entries, 2962 to 4602791
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   YEAR_DESC                    77559 non-null  int64  
 1   REGION_NAME                  77559 non-null  object 
 2   REGION_CODE                  77559 non-null  object 
 3   GEOGRAPHY_NAME               77559 non-null  object 
 4   GEOGRAPHY_CODE               77559 non-null  object 
 5   BNF_PRESENTATION_CODE        77559 non-null  object 
 6   BNF_PRESENTATION_NAME        77559 non-null  object 
 7   SNOMED_CODE                  77559 non-null  float64
 8   SUPPLIER_NAME                56064 non-null  object 
 9   UNIT_OF_MEASURE              77559 non-null  object 
 10  GENERIC_BNF_EQUIVALENT_CODE  77559 non-null  object 
 11  GENERIC_BNF_EQUIVALENT_NAME  77559 non-null  object 
 12  BNF_CHEMICAL_SUBSTANCE_CODE  77559 non-null  object 
 13  BNF_CHEMICAL_SUB

In [26]:
df_ad.describe()

,YEAR_DESC,SNOMED_CODE,BNF_PARAGRAPH_CODE,BNF_SECTION_CODE,BNF_CHAPTER_CODE,PREP_CLASS,PRESCRIBED_PREP_CLASS,ITEMS,TOTAL_QUANTITY,NIC,NIC_PER_ITEM,NIC_PER_QUANTITY,QUANTITY_PER_ITEM
count,77559.000000,7.755900e+04,77559.000000,77559.0,77559.0,77559.000000,77559.000000,7.755900e+04,7.755900e+04,7.755900e+04,77559.000000,77559.000000,77559.000000
mean,2021.466058,1.833438e+16,40302.846388,403.0,4.0,3.296961,3.017264,6.476273e+03,2.399108e+05,1.964798e+04,45.790074,0.628947,63.081322
std,1.714522,1.537480e+16,1.159710,0.0,0.0,1.645198,1.719709,3.554905e+04,1.358158e+06,9.404161e+04,140.866972,1.438870,76.826989
min,2019.000000,3.217450e+08,40301.000000,403.0,4.0,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000e-02,0.010000,0.000360,1.000000
25%,2020.000000,7.690110e+14,40302.000000,403.0,4.0,1.000000,1.000000,3.000000e+00,1.680000e+02,3.400000e+01,2.694775,0.071078,28.000000
50%,2021.000000,1.537571e+16,40303.000000,403.0,4.0,3.000000,3.000000,2.400000e+01,1.260000e+03,4.411200e+02,9.500000,0.189480,37.333330
75%,2023.000000,3.474091e+16,40304.000000,403.0,4.0,5.000000,5.000000,3.990000e+02,1.903100e+04,7.603890e+03,28.141255,0.667860,58.536950
max,2024.000000,4.378771e+16,40304.000000,403.0,4.0,5.000000,5.000000,1.023205e+06,4.692442e+07,6.326114e+06,4794.000000,68.235000,2250.000000


### 2.2 Data Cleaning

#### Null Handling

    SUPPLIER_NAME
The column "SUPPLIER_NAME" is the only column with missing values. It has <b>21,495 null values</b>, which accounts for roughly <b>27.7% of all records</b>. This is quite a significant amount of missing data.

Data on the supplier names for what antidepressants will become important when analysing the <b>market share of suppliers</b> and seeing who are producing the most antidepressant prescriptions.

21,495 would be a very significant number of rows to drop, so I'd prefer not to just drop these rows from the dataset.

Instead, I will <b>replace these null values</b> with a string <b>"Unknown"</b>.

The benefit of this approach is that I can <b>retain the data I do have for these rows</b>, and also when it comes to analysis of market share, I can apply a <b>filter</b> in Tableau to <b>exclude any records with a supplier name of "Unknown"</b>. 

In [27]:
# Impute any missing values in SUPPLIER_NAME with "Unknown"
df_ad['SUPPLIER_NAME'].fillna('Unknown', inplace=True)

/var/folders/gv/ry8fcb9x2753_978l6fnrq640000gn/T/ipykernel_4751/1588970095.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_ad['SUPPLIER_NAME'].fillna('Unknown', inplace=True)


In [28]:
# Check to see if it's worked
df_ad['SUPPLIER_NAME'].isnull().sum()

0

Looks good!

#### Data Casting

In [29]:
df_ad.dtypes;

SNOMED_CODE is currently being stored as a float64 and is being displayed in scientific notation. Snomed codes are numerical codes that the NHS use to define various clinical characteristics of prescriptions for data processing purposes. They should be stored as strings. I will convert them to type string.

This is important in case I join this dataset to another. SNOMED codes are a standardised indentifier of clinical information, so this must be retained in order to join to other datasets.

In [30]:
# If I were to convert to string straight from float, the scientific notation would be retained which is not helpful.
# First, I will convert SNOMED_CODE to type int. This will display the full numerical code and remove its scientific notation display.
# THEN, I will convert to string. This should properly display each SNOMED code, in its numerical format, as a string.

# Converts to type int
df_ad['SNOMED_CODE'] = df_ad['SNOMED_CODE'].astype(int)

# Converts to type string
df_ad['SNOMED_CODE'] = df_ad['SNOMED_CODE'].astype(str)

In [31]:
# Check to see it's worked
df_ad['SNOMED_CODE'].head()

2962    13879011000001108
2963    35901611000001104
2964      563711000001107
2965            321745007
2966      310911000001104
Name: SNOMED_CODE, dtype: object

That's fixed!

All other columns look good.

#### Check's for String Consistency and Cardinality

Let's check for any inconsistencies in how strings are being stored. This is to make sure the same information isn't being stored differently (i.e. Geography_name containing both "North London" AND "northlondon". We want these to be treated as the same geography name).

In [32]:
df_ad['GEOGRAPHY_CODE'].nunique()

42

In [33]:
df_ad['GEOGRAPHY_NAME'].nunique()

84

We can see that there are 42 Geography codes in our dataset - codes that represent key regions that the NHS operates in as individual bodies.

<b> There are 42 Codes, but 84 names - why?</b>

First, let's have a look at which codes have more than 1 name associated with it.

As the number of names is double the number of codes, it is most likely that each code has 2 names associated with it, but it' best to check.

In [34]:
# Group by GEOGRAPHY_CODE and list unique names associated with each code
code_to_names_mapping = df_ad.groupby('GEOGRAPHY_CODE')['GEOGRAPHY_NAME'].unique()

# Filter to see only codes that have more than one unique name
codes_with_multiple_names = code_to_names_mapping[code_to_names_mapping.apply(len) > 1]

codes_with_multiple_names.count()


42

As we can see, all 42 codes have more than one name associated with it.

<b>Why?</b>

This is because, in 2022, the regional bodies of the NHS that these codes/names represent changed from "Sustainability and Transformation Partnerships" (STP's) to "Integrated Care Boards" (ICB's).
This mostly involved various regulatory changes.

It also meant changes in the name of that body, and in a small number of cases, minor changes in the regional borders that that body provides support for (this change is not significant enough to disrupt the explanatory power of this analysis).

For consistency when it comes to visualising this data, it is best to retain only one name for each code.

As ICB's are now the standard, we will use the ICB names for all data from 2019-2024. This will improve the ability to compare between years and regions during EDA.

In [35]:
# In order to replace the STP names, I first need to get a dictionary that contains all of the ICB names for each ICB code. These can be found y using the most recent year in the dataset.

# Create a DataFrame with each geography code and their names from the latest year (the ICB names)
latest_names_per_code = df_ad.loc[df_ad.groupby('GEOGRAPHY_CODE')['YEAR_DESC'].idxmax()][['GEOGRAPHY_CODE', 'GEOGRAPHY_NAME']]
latest_names_per_code = latest_names_per_code.drop_duplicates(subset=['GEOGRAPHY_CODE'])

# Convert this to a dictionary
definitive_code_to_name_map = pd.Series(latest_names_per_code.GEOGRAPHY_NAME.values, index=latest_names_per_code.GEOGRAPHY_CODE).to_dict()

# Check that it's worked
definitive_code_to_name_map


{'QE1': 'NHS LANCASHIRE AND SOUTH CUMBRIA INTEGRATED CARE BOARD',
 'QF7': 'NHS SOUTH YORKSHIRE INTEGRATED CARE BOARD',
 'QGH': 'NHS HEREFORDSHIRE AND WORCESTERSHIRE INTEGRATED CARE BOARD',
 'QH8': 'NHS MID AND SOUTH ESSEX INTEGRATED CARE BOARD',
 'QHG': 'NHS BEDFORDSHIRE, LUTON AND MILTON KEYNES INTEGRATED CARE BOARD',
 'QHL': 'NHS BIRMINGHAM AND SOLIHULL INTEGRATED CARE BOARD',
 'QHM': 'NHS NORTH EAST AND NORTH CUMBRIA INTEGRATED CARE BOARD',
 'QJ2': 'NHS DERBY AND DERBYSHIRE INTEGRATED CARE BOARD',
 'QJG': 'NHS SUFFOLK AND NORTH EAST ESSEX INTEGRATED CARE BOARD',
 'QJK': 'NHS DEVON INTEGRATED CARE BOARD',
 'QJM': 'NHS LINCOLNSHIRE INTEGRATED CARE BOARD',
 'QK1': 'NHS LEICESTER, LEICESTERSHIRE AND RUTLAND INTEGRATED CARE BOARD',
 'QKK': 'NHS SOUTH EAST LONDON INTEGRATED CARE BOARD',
 'QKS': 'NHS KENT AND MEDWAY INTEGRATED CARE BOARD',
 'QM7': 'NHS HERTFORDSHIRE AND WEST ESSEX INTEGRATED CARE BOARD',
 'QMF': 'NHS NORTH EAST LONDON INTEGRATED CARE BOARD',
 'QMJ': 'NHS NORTH CENTRAL LOND

In [36]:
# Now I will map the ICB names to all their corresponding geography codes, replacing the old STP geography names
df_ad['GEOGRAPHY_NAME'] = df_ad['GEOGRAPHY_CODE'].map(definitive_code_to_name_map)
df_ad.head();


In [37]:
df_ad['GEOGRAPHY_NAME'].nunique()

42

<b>Okay, the geography names are now standardised to display only their ICB version!</b>

Let's do a quick check for any <b>consistency/cardinality</b> issues for <b>other key columns</b>.

In [38]:
df_ad['REGION_NAME'].unique()

array(['LONDON', 'SOUTH WEST', 'SOUTH EAST', 'MIDLANDS',
       'EAST OF ENGLAND', 'NORTH WEST', 'NORTH EAST AND YORKSHIRE'],
      dtype=object)

In [39]:
df_ad['BNF_CHEMICAL_SUBSTANCE'].unique();

In [40]:
df_ad['BNF_PARAGRAPH'].unique()

array(['Tricyclic and related antidepressant drugs',
       'Monoamine-oxidase inhibitors (maois)',
       'Selective serotonin re-uptake inhibitors',
       'Other antidepressant drugs'], dtype=object)

In [41]:
df_ad['UNIT_OF_MEASURE'].unique()

array(['ml', 'tablet', 'capsule', 'UNKNOWN'], dtype=object)

All looks good!

#### The data is now looking much cleaner and ready to use!

### 2.3 Exporting into Tableau

Much of the EDA and data visualisation will be completed in Tableau. I will export this df_ad as a .csv file to import into Tableau!

Once the dataset is inside Tableau, there is one more thing I need to do before I start EDA.

* This dataset contains columns based on geographical locations. However, these geographical locations, called ICB's, are not recognised by Tableau as their borders do not conform to any standard geographical borders such as for countries/regions/states.


* In order to visualise this data geographically (on a map, for example), I must join this dataset with a shapefile I have downloaded that contains the information needed for Tableau to understand the geographical borders for each ICB. This will allow the data to be visualised on a map.


* Once I have connected these two together, I will start on EDA in Tableau.

#### Connecting the dataset to the ICB shapefile in tableau

After inspecting the shapefile I have for all the ICB's, can see that the strings contained in the column I will use to join the shapefile to df_ad (I am joining them by geography name) do not match exactly.

* In df_ad, they are written in capital letters

* In the shapefile, however, they are in title case.

To resolve this, I will first convert df_ad['GEOGRAPHY_NAME'] to title case, and THEN export it as a .csv file to load into Tableau.

In [42]:
# This converts each ICB name to Title case
df_ad['GEOGRAPHY_NAME'] = df_ad['GEOGRAPHY_NAME'].str.title()

# "NHS", however needs to remain capitalised. This code uses RegEx to replace the now title-cased "Nhs" with "NHS"
df_ad['GEOGRAPHY_NAME'] = df_ad['GEOGRAPHY_NAME'].str.replace(r'^Nhs\b', 'NHS', regex=True)

# In the shapefile, the word "and" is also not title-cased, so I need to fix that too
df_ad['GEOGRAPHY_NAME'] = df_ad['GEOGRAPHY_NAME'].str.replace(r'\bAnd\b', 'and', regex=True)

# The same for the word "the"
df_ad['GEOGRAPHY_NAME'] = df_ad['GEOGRAPHY_NAME'].str.replace(r'\bThe\b', 'the', regex=True)

# The same for the word "of"
df_ad['GEOGRAPHY_NAME'] = df_ad['GEOGRAPHY_NAME'].str.replace(r'\bOf\b', 'of', regex=True)

# The same for the word "of" in "Stoke-on-Trent"
df_ad['GEOGRAPHY_NAME'] = df_ad['GEOGRAPHY_NAME'].str.replace(r'\b-On-\b', '-on-', regex=True)

In [ ]:
# Export the dataframe to a CSV file
# This will be loaded into Tableau for visualisation
df_ad.to_csv('antidepressant_pca.csv', index=False)

#### Done! The dataset is now in Tableau and I have connected it to its shapefile

## 3. EDA